# Data Cleaning and Pre-Processing

### Import libraries needed

In [1]:
import pandas as pd
import json
import requests
from shapely.geometry import Point
from shapely.geometry import shape
from bs4 import BeautifulSoup
import geopandas as gpd
import numpy as np
from sklearn.cluster import KMeans

Credentials for map search later

In [2]:
# Foursquare credentials
with open('credentials.json') as f:
    creds = json.load(f)

    CLIENT_ID = creds[0]['CLIENT_ID']
    CLIENT_SECRET = creds[0]['CLIENT_SECRET']

VERSION =  "20200101"
LIMIT = 100

## Crime Dataframe

In [3]:
crime_df = pd.read_csv('MPS Borough Level Crime (most recent 24 months).csv')
crime_df.head()

,MajorText,MinorText,LookUp_BoroughName,201812,201901,201902,201903,201904,201905,201906,...,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011
0,Arson and Criminal Damage,Arson,Barking and Dagenham,1,5,2,5,5,11,3,...,5,6,2,2,4,4,6,2,7,4
1,Arson and Criminal Damage,Criminal Damage,Barking and Dagenham,88,97,127,138,130,140,113,...,103,107,80,86,121,122,114,116,119,100
2,Burglary,Burglary - Business and Community,Barking and Dagenham,33,45,24,29,27,21,27,...,17,28,29,16,16,28,24,32,21,19
3,Burglary,Burglary - Residential,Barking and Dagenham,164,114,107,99,96,114,96,...,123,97,57,42,63,72,63,54,67,90
4,Drug Offences,Drug Trafficking,Barking and Dagenham,4,6,2,6,5,9,6,...,6,6,15,13,12,21,9,11,14,17


In [4]:
crime_df['Total_Crime'] = crime_df.sum(axis=1)

In [5]:
crime_df = crime_df[['MajorText', 'LookUp_BoroughName', 'Total_Crime']]
crime_df.columns = ['Crime', 'Borough', 'Total_Crime']
crime_df.head()

,Crime,Borough,Total_Crime
0,Arson and Criminal Damage,Barking and Dagenham,115
1,Arson and Criminal Damage,Barking and Dagenham,2687
2,Burglary,Barking and Dagenham,656
3,Burglary,Barking and Dagenham,2193
4,Drug Offences,Barking and Dagenham,228


In [6]:
# get a look at all the different crime types
crime_df.Crime.unique()

array(['Arson and Criminal Damage', 'Burglary', 'Drug Offences',
       'Miscellaneous Crimes Against Society', 'Possession of Weapons',
       'Public Order Offences', 'Robbery', 'Sexual Offences', 'Theft',
       'Vehicle Offences', 'Violence Against the Person'], dtype=object)

In [7]:
type_crime_df = crime_df.groupby(['Borough', 'Crime']).sum()
type_crime_df.head()

Total_Crime
Borough              Crime                                            
Barking and Dagenham Arson and Criminal Damage                    2802
                     Burglary                                     2849
                     Drug Offences                                2768
                     Miscellaneous Crimes Against Society          633
                     Possession of Weapons                         353

In [8]:
total_crime_df = crime_df.groupby('Borough').sum()
total_crime_df.head()

,Total_Crime
Borough,
Barking and Dagenham,39456
Barnet,59167
Bexley,33929
Brent,59257
Bromley,47499


## Borough Dataframe

In [9]:
# get boroughs from wikipedia
url = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
wikitable = "wikitable sortable jquery-tablesorter"
response = requests.get(url)
print(response.status_code)  # check connection

200


In [10]:
# nb. ignoring 2nd table on wiki, don't have geodata for city
soup = BeautifulSoup(response.text, 'html.parser')
london = soup.find('table', {'class': "wikitable"})

london_df = pd.read_html(str(london))  # first table
london_df = pd.DataFrame(london_df[0])
london_df.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2019 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,212906,".mw-parser-output .geo-default,.mw-parser-outp...",25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,395896,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,248287,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,329771,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,332336,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


In [11]:
# remove unnecessary columns, rename columns
# has the coords, but they aren't in the centre of the borough, so the searches don't work!
london_df = london_df[['Borough','Population (2019 est)[1]']]
london_df.columns = ['Borough','Population']
# get rid of the [note n] notes
london_df.Borough = london_df['Borough'].replace(to_replace =" \[note [1-9]\]", value = '', regex = True)
london_df.head()

,Borough,Population
0,Barking and Dagenham,212906
1,Barnet,395896
2,Bexley,248287
3,Brent,329771
4,Bromley,332336


In [12]:
# get the coords for each borough
coords = pd.read_csv('Local_Authority_Districts.csv')
coords = coords[['LAD19NM', 'LONG', 'LAT']]
coords.columns = ['Borough', 'longitude', 'latitude']
coords.head()

,Borough,longitude,latitude
0,Hartlepool,-1.27023,54.6762
1,Middlesbrough,-1.21099,54.5447
2,Redcar and Cleveland,-1.00611,54.5675
3,Stockton-on-Tees,-1.30669,54.5569
4,Darlington,-1.56835,54.5353


In [13]:
london_df = pd.merge(london_df, coords, on='Borough')
london_df.head()

,Borough,Population,longitude,latitude
0,Barking and Dagenham,212906,0.129506,51.5455
1,Barnet,395896,-0.218210,51.6111
2,Bexley,248287,0.146212,51.4582
3,Brent,329771,-0.275680,51.5644
4,Bromley,332336,0.039246,51.3727


In [14]:
# add total crimes for each borough
london_crimes = pd.merge(london_df, total_crime_df, on='Borough')
london_crimes.head()

,Borough,Population,longitude,latitude,Total_Crime
0,Barking and Dagenham,212906,0.129506,51.5455,39456
1,Barnet,395896,-0.218210,51.6111,59167
2,Bexley,248287,0.146212,51.4582,33929
3,Brent,329771,-0.275680,51.5644,59257
4,Bromley,332336,0.039246,51.3727,47499


In [15]:
# calculate crime rate for each borough
london_crimes['Crime_Rate'] = round((london_crimes['Total_Crime'] / london_crimes['Population']) * 100000, 2)
london_crimes.head()

,Borough,Population,longitude,latitude,Total_Crime,Crime_Rate
0,Barking and Dagenham,212906,0.129506,51.5455,39456,18532.12
1,Barnet,395896,-0.218210,51.6111,59167,14945.09
2,Bexley,248287,0.146212,51.4582,33929,13665.23
3,Brent,329771,-0.275680,51.5644,59257,17969.14
4,Bromley,332336,0.039246,51.3727,47499,14292.46


In [16]:
bin_labels = ['Low', 'Medium', 'High']
london_crimes['Crime_Rate_binned'] = pd.qcut(london_crimes['Crime_Rate'], q=3, labels=bin_labels)
london_crimes.head()

,Borough,Population,longitude,latitude,Total_Crime,Crime_Rate,Crime_Rate_binned
0,Barking and Dagenham,212906,0.129506,51.5455,39456,18532.12,Medium
1,Barnet,395896,-0.218210,51.6111,59167,14945.09,Low
2,Bexley,248287,0.146212,51.4582,33929,13665.23,Low
3,Brent,329771,-0.275680,51.5644,59257,17969.14,Medium
4,Bromley,332336,0.039246,51.3727,47499,14292.46,Low


geojson

In [17]:
london_geo = gpd.read_file('gb_boroughs.json')
london_geo.head()

,LAD13CD,LAD13CDO,LAD13NM,LAD13NMW,geometry
0,E06000001,00EB,Hartlepool,None,"MULTIPOLYGON (((-1.24196 54.72245, -1.24251 54..."
1,E06000002,00EC,Middlesbrough,None,"MULTIPOLYGON (((-1.19705 54.58178, -1.19436 54..."
2,E06000003,00EE,Redcar and Cleveland,None,"MULTIPOLYGON (((-1.20099 54.57759, -1.20024 54..."
3,E06000004,00EF,Stockton-on-Tees,None,"MULTIPOLYGON (((-1.30387 54.56514, -1.30199 54..."
4,E06000005,00EH,Darlington,None,"POLYGON ((-1.63799 54.61719, -1.63766 54.61669..."


In [18]:
# the names are weird and can't be found when i search for them so...
london_geo.columns = ['dunno', 'stuff', 'Borough', 'huh', 'geometry']
london_geo = london_geo[['Borough', 'geometry']]
london_geo = london_crimes.merge(london_geo, on='Borough')
london_geo.tail()

,Borough,Population,longitude,latitude,Total_Crime,Crime_Rate,Crime_Rate_binned,geometry
27,Sutton,206349,-0.17227,51.3576,27146,13155.38,Low,"POLYGON ((-0.16646 51.38969, -0.16529 51.38798..."
28,Tower Hamlets,324745,-0.03640,51.5155,67146,20676.53,High,"POLYGON ((-0.02947 51.54303, -0.02901 51.54227..."
29,Waltham Forest,276983,-0.01881,51.5946,47946,17310.09,Medium,"POLYGON ((-0.00822 51.64361, -0.00668 51.64333..."
30,Wandsworth,329677,-0.20021,51.4524,51171,15521.56,Low,"POLYGON ((-0.12636 51.48447, -0.12750 51.48202..."
31,Westminster,261317,-0.15295,51.5122,134848,51603.23,High,"POLYGON ((-0.17414 51.53820, -0.17351 51.53763..."


## Get venues

In [19]:
# function for checking something is in the right borough
def check_venue(borough, lat, lon):
        point = Point(lon, lat)
        poly = london_geo[london_geo['Borough'] == borough]['geometry'].item()
        poly = shape(poly)

        if poly.contains(point):
            return True
        else:
            return False


# function for getting venues
def get_venues(boroughs, lats, lngs, cat):
    radius = 10000

    venues_list=[]
    for borough, lat, lng in zip(boroughs, lats, lngs):
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT,
            cat
            )

        results = requests.get(url).json()["response"]['venues']

        if check_venue(borough, lat, lng):
            venues_list.append([(
                borough,
                lat,
                lng,
                v['name'],
                v['location']['lat'],
                v['location']['lng'],
                v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough',
                             'Latitude',
                             'Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']

    return nearby_venues

In [20]:
# different categories of venues
cat_art = '4d4b7104d754a06370d81259'  # Arts & Entertainment
cat_out = '4d4b7105d754a06377d81259' # outdoors recreation
cat_study = '4d4b7105d754a06372d81259,4bf58dd8d48988d12f941735' # college and university, libraries
cat_social = '52e81612bcbc57f1066b7a34,52e81612bcbc57f1066b7a33,4bf58dd8d48988d131941735'  # community centre, social club, religious centres

# values for getting venues
bor = london_crimes.Borough
lat = london_crimes.latitude
lng = london_crimes.longitude

arts = get_venues(bor, lat, lng, cat_art)
outs = get_venues(bor, lat, lng, cat_out)
studs = get_venues(bor, lat, lng, cat_study)
socials = get_venues(bor, lat, lng, cat_social)

In [21]:
studs.head()

,Borough,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barking and Dagenham,51.5455,0.129506,Dagenham Library,51.540330,0.148273,Library
1,Barking and Dagenham,51.5455,0.129506,Powerleague Newham,51.525194,0.078000,College Football Field
2,Barking and Dagenham,51.5455,0.129506,The Green,51.544845,0.011269,College Cafeteria
3,Barking and Dagenham,51.5455,0.129506,The Point,51.502878,0.040015,General College & University
4,Barking and Dagenham,51.5455,0.129506,Powerleague Fairlop,51.600130,0.102691,College Football Field


In [22]:
arts_total = arts.groupby('Borough').count()
outs_total = outs.groupby('Borough').count()
studs_total = studs.groupby('Borough').count()
socials_total = socials.groupby('Borough').count()

In [23]:
arts_total.head()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Barking and Dagenham,50,50,50,50,50,50
Barnet,49,49,49,49,49,49
Bexley,47,47,47,47,47,47
Brent,49,49,49,49,49,49
Bromley,46,46,46,46,46,46


In [24]:
# add them to london_crimes df
london1 = london_crimes.merge(arts_total['Venue'], on='Borough')
london1.rename(columns = {'Venue':'Arts'}, inplace=True)
london1 = london1.merge(outs_total['Venue'], on='Borough')
london1.rename(columns = {'Venue':'Outdoors'}, inplace=True)
london1 = london1.merge(studs_total['Venue'], on='Borough')
london1.rename(columns = {'Venue':'Study'}, inplace=True)
london1 = london1.merge(socials_total['Venue'], on='Borough')
london1.rename(columns = {'Venue':'Social and Religious'}, inplace=True)
london1.head()

,Borough,Population,longitude,latitude,Total_Crime,Crime_Rate,Crime_Rate_binned,Arts,Outdoors,Study,Social and Religious
0,Barking and Dagenham,212906,0.129506,51.5455,39456,18532.12,Medium,50,42,48,49
1,Barnet,395896,-0.218210,51.6111,59167,14945.09,Low,49,32,50,50
2,Bexley,248287,0.146212,51.4582,33929,13665.23,Low,47,40,44,48
3,Brent,329771,-0.275680,51.5644,59257,17969.14,Medium,49,37,50,50
4,Bromley,332336,0.039246,51.3727,47499,14292.46,Low,46,40,45,48


In [26]:
# get total venues
london1['Total_Venues'] = london1.Arts + london1.Outdoors + london1.Study + london1['Social and Religious']
london1.to_csv('london1.csv', index=False)

## now to see what clustering comes up with

In [27]:
all_venues = pd.concat([arts, outs, studs, socials])

In [28]:
# lets see how k-means clustering sorts them and if that correlates
# one hot encoding
london_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Borough'] = all_venues['Borough']

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Borough,Adult Education Center,American Restaurant,Animal Shelter,Arcade,Art Gallery,Art Museum,Arts & Entertainment,Athletics & Sports,BBQ Joint,...,Theme Park Ride / Attraction,Track Stadium,Trade School,Trail,University,Water Park,Waterfront,Well,Zoo,Zoo Exhibit
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Barking and Dagenham,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
london_grouped = london_onehot.groupby('Borough').mean().reset_index()
london_grouped

,Borough,Adult Education Center,American Restaurant,Animal Shelter,Arcade,Art Gallery,Art Museum,Arts & Entertainment,Athletics & Sports,BBQ Joint,...,Theme Park Ride / Attraction,Track Stadium,Trade School,Trail,University,Water Park,Waterfront,Well,Zoo,Zoo Exhibit
0,Barking and Dagenham,0.000000,0.000000,0.000000,0.005291,0.047619,0.000000,0.000000,0.010582,0.000000,...,0.000000,0.000000,0.000000,0.005291,0.026455,0.000000,0.000000,0.000000,0.000000,0.005291
1,Barnet,0.000000,0.000000,0.000000,0.000000,0.016575,0.000000,0.000000,0.005525,0.000000,...,0.000000,0.005525,0.000000,0.005525,0.038674,0.000000,0.000000,0.000000,0.011050,0.011050
2,Bexley,0.000000,0.000000,0.000000,0.000000,0.016760,0.000000,0.000000,0.005587,0.000000,...,0.000000,0.000000,0.000000,0.005587,0.022346,0.005587,0.000000,0.000000,0.005587,0.011173
3,Brent,0.000000,0.005376,0.000000,0.000000,0.032258,0.010753,0.000000,0.010753,0.000000,...,0.000000,0.005376,0.000000,0.000000,0.043011,0.000000,0.000000,0.000000,0.005376,0.005376
4,Bromley,0.000000,0.000000,0.005587,0.005587,0.022346,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.016760,0.000000,0.000000,0.000000,0.000000,0.000000
5,Camden,0.005348,0.005348,0.000000,0.000000,0.016043,0.032086,0.000000,0.005348,0.000000,...,0.000000,0.005348,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.005348,0.005348
6,Croydon,0.000000,0.000000,0.000000,0.000000,0.021505,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.005376,0.000000,0.010753,0.000000,0.000000,0.000000,0.000000,0.000000
7,Ealing,0.000000,0.005319,0.000000,0.000000,0.031915,0.010638,0.000000,0.010638,0.000000,...,0.000000,0.005319,0.000000,0.000000,0.015957,0.000000,0.000000,0.000000,0.000000,0.000000
8,Enfield,0.000000,0.000000,0.000000,0.000000,0.032432,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.005405,0.016216,0.000000,0.005405,0.000000,0.000000,0.000000,0.005405
9,Greenwich,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.005291,0.052910,0.000000,0.000000,0.000000,0.000000,0.005291


In [30]:
# top 10 categories per borough

# function to find the top venues for each neighbourhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Boroughs_venues_sorted = pd.DataFrame(columns=columns)
Boroughs_venues_sorted['Borough'] = london_grouped['Borough']

for ind in np.arange(london_grouped.shape[0]):
    Boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

Boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Church,Park,Community Center,Library,Art Gallery,College Academic Building,General College & University,Historic Site,College Classroom,University
1,Barnet,Church,Park,Library,University,History Museum,College Academic Building,Historic Site,College Residence Hall,General College & University,Pub
2,Bexley,Church,Park,Library,Historic Site,College Academic Building,Community Center,College Classroom,Gym / Fitness Center,Social Club,University
3,Brent,Church,Park,Library,University,Art Gallery,College Academic Building,Historic Site,Soccer Stadium,Bridge,Social Club
4,Bromley,Church,Park,Library,Community College,Gym / Fitness Center,College Academic Building,Historic Site,Art Gallery,General Entertainment,General College & University


In [31]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = london_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([2, 1, 2, 1, 3, 0, 3, 1, 2, 2])

In [32]:
# add clustering labels
Boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london2= london_crimes.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each Borough
london2 = london2.join(Boroughs_venues_sorted.set_index('Borough'), on='Borough')

london2.head()

,Borough,Population,longitude,latitude,Total_Crime,Crime_Rate,Crime_Rate_binned,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,212906,0.129506,51.5455,39456,18532.12,Medium,2,Church,Park,Community Center,Library,Art Gallery,College Academic Building,General College & University,Historic Site,College Classroom,University
1,Barnet,395896,-0.218210,51.6111,59167,14945.09,Low,1,Church,Park,Library,University,History Museum,College Academic Building,Historic Site,College Residence Hall,General College & University,Pub
2,Bexley,248287,0.146212,51.4582,33929,13665.23,Low,2,Church,Park,Library,Historic Site,College Academic Building,Community Center,College Classroom,Gym / Fitness Center,Social Club,University
3,Brent,329771,-0.275680,51.5644,59257,17969.14,Medium,1,Church,Park,Library,University,Art Gallery,College Academic Building,Historic Site,Soccer Stadium,Bridge,Social Club
4,Bromley,332336,0.039246,51.3727,47499,14292.46,Low,3,Church,Park,Library,Community College,Gym / Fitness Center,College Academic Building,Historic Site,Art Gallery,General Entertainment,General College & University


In [33]:
london2.to_csv('london2.csv', index=False)